In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm_notebook as tqdm

In [2]:
today = datetime.today()
date_list = [(today - timedelta(days = x)).strftime("%Y%m%d") for x in range(10)]

In [3]:
origo_dict = {}
for date in tqdm(date_list):
    response = requests.get(f'https://www.origo.hu/hir-archivum/2019/{date}.html')
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = [article.get("href") for article in soup.select(".archive-cikk a")]
    for article in articles:
        response = requests.get(article)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            sources = [source.text.split("Forrás: ")[1] for source in soup.select(".source")]
        except:
            sources = []
        try:
            address_top = soup.find('div', class_='address top')
            author = address_top.find("span", {"class":"article-author"}).text
            date = address_top.find("span", {"id":"article-date"}).text
        except:
            author = None
            date = None

        paragraphs = soup.select("#article-text p")
        related_urls = []
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    if "http" in link.get('href'):
                        related_urls.append(link.get('href'))
                    else:
                        related_urls.append("https:" + link.get('href'))
        origo_dict[article] = {
            "related_links" : related_urls,
            "sources" : sources,
            "author" : author,
            "publish_date" : date
            #"last_modification_date" : dik[0]["_mod_ts"]
        }

In [4]:
origo_dict

{'https://www.origo.hu/auto/20191219-gondolta-besorol-a-kamionos-ele-az-nem-kifejezes-hogy-nem-kellett-volna.html': {'related_links': [],
  'sources': [],
  'author': 'Origo',
  'publish_date': '2019.12.19. 09:58'},
 'https://www.origo.hu/auto/20191219-vw-golf-8-toresteszt.html': {'related_links': [],
  'sources': ['Euro NCAP'],
  'author': 'Origo',
  'publish_date': '2019.12.19. 11:27'},
 'https://www.origo.hu/auto/20191219-megkoszonte-hogy-atengedtek-ezutan-gazoltak-halalra-a-tatai-zebran.html': {'related_links': ['https://www.origo.hu/itthon/20191218-kijelolt-gyalogosatkelohelyen-gazoltak-halalra-egy-ferfit-tatan-kedden-este.html',
   'https://www.nyugat.hu/cikk/korulnezett_integetett_mosolygott_aztan_halalra'],
  'sources': [],
  'author': 'Origo',
  'publish_date': '2019.12.19. 13:00'},
 'https://www.origo.hu/auto/20191219-kiderult-valami-a-magyar-autosokrol-aminek-nagyon-nem-fognak-orulni-a-gyartok.html': {'related_links': [],
  'sources': [],
  'author': 'Origo',
  'publish_date